# 面向对象导论

## 面向对象基础

抽象、封装、多态、继承

其中封装是对于实例变量而言的，其保存了对象的状态。多态是由于继承复用代码带来的一种调用超类的接口的契约保证。

## 面向对象设计原则

设计模式的任务是设计出弹性的，可以维护的，能够应对变化的代码，这仅靠抽象，继承，多态这些OOP观念是不足以做到的。

所有的设计模式都遵循以下的设计原则：

- 找出应用中可能需要变化之处，把它们独立出来，不要和那些不需要变化的代码混在一起。换句话说，封装变化。
- 针对接口（超类型）编程，而不是针对实现编程
- 多用组合，少用继承（组合可以将代码被封装成类，可以在运行时动态改变行为）
- 类和类之间尽可能的解耦，使用尽可能少的方法传递信息

良好的面向对象设计应该满足 复用（使用组合或者继承）、可扩充（扩充接口）、可维护（多态，针对接口编程）三大特征。

# 策略模式

**策略模式定义了算法族，分别封装起来，让它们之间可以互相替换，此模式让算法的变化独立于使用算法的客户。**

策略模式本身就是组合，其对应has-a关系，使用实例变量来保存状态，针对接口编程来调用和更改这个变量以改变状态。JDBC就是典型的策略模式。

# 观察者模式

**观察者模式定义了对象之间的一对多依赖，当一个对象改变状态时，它的所有依赖者都会收到通知并且自动更新。**主题->观察者 = 发布者->订阅者模式，一个发布者，多个订阅者，订阅者可以选择进入或者退出发布者名单，这样就不会收到更新。

常见的观察者模式是GUI，JavaBeans，RMI

## 自己实现观察者模式

常用Subject和Observer表示类名。

其中Subject应该保存一个注册者列表，提供添加和删除注册者的方法，提供更新注册者数据的方法，并且将自己的状态通过注册者的接口传递给注册者。

对于注册者而言，应该提供一个接受发布者状态的接口，应该有保存这些状态的实例，同时最好有发布者的一个实例，方便和发布者绑定或者取消订阅。

**几乎完全解耦 **观察者模式类似于集体主义，一个对象保留并且获取状态，而其余对象则建立和这个对象的联系，并且从其中获取状态。由于两者间传递信息仅使用了注册者update接口，除此之外，发布者保存了含有注册者引用的对象，而这个对象的保存是依靠注册者传入发布者的一个实例，调用add添加的，因此除了注册者update，发布者的add，remove之外，完全做到了解耦。发布者自动进入注册者内存并且更新其状态，这个过程不需要注册者任何操作。

## 使用util实现观察者模式

```java
import java.util.Observable;
import java.util.Observer;

public class Main {
    public static void main(String[] args) {
        Server server = new Server();
        ClientA clientA = new ClientA(server);
        server.setStatic(10.0,23.0);
        server.notice();
    }
}
class Server extends Observable {
    private double temp;
    private double pressure;
    public double getTemp() {
        return temp;
    }
    public double getPressure() {
        return pressure;
    }
    public void setStatic(double temp, double pressure) {
        this.temp = temp;
        this.pressure = pressure;
    }
    public void notice() {
        this.setChanged();
        this.notifyObservers();
    }
}
class ClientA implements Observer {
    private double temp;
    private double pressure;
    private Observable server;
    ClientA(Observable server) {
        this.server = server;
        this.server.addObserver(this);
    }
    @Override
    public void update(Observable o, Object arg) {
        temp = ((Server)o).getTemp();
        pressure = ((Server)o).getPressure();
        display();
    }
    public void display() {
        System.out.println("Current temp is " + temp + ". Current Pressure is " + pressure);
    }
}
```

上述代码是使用观察者的一个例子，类似的，在Swing以及JavaFx中，存在大量的这种观察者设计模式：listener。

注意上述的惯用方法，对于观察对象而言，其主做的就是从自身保有的列表中获取观察者，然后调用Observaer的update方法，将自己传递过去。我们主要的任务就是实现一些需要在传送之后用于获取Observable状态的方法，**getXXX()**，然后**setChanged并且notifyObservers**即可。

对于观察者而言，我们需要选择一个观察对象，**注册它**或者从它的名单中删除。此外，在**update方法**中决定当收到观察对象更新时的自己的操作。除此之外，不需要做任何事情，当观察对象有更新，其会自动调用update改变观察者的状态。

## 观察者模式的设计原则

- 封装变化和不变的部分：观察者模式中，变化的部分是Observable对象，因此将其单独封装为一个类，然后将所有Observer的这些状态抽取出来，接受更新
- 针对接口而不是实现编程：我们使用了update方法来更新观察者状态，使用addObserver来进行观察对象的注册。需要注意，可以寻求对于getXXXX这种需要在观察者update方法进行实现的接口进行标准化，使用interface达成契约。这样的话，Observer只和Observable以及这个获取具体状态的契约接口耦合。Observable只和这个Observer的update方法耦合。
- 多用组合，少用继承：没错，我们在Observer中保存了Observable的实例，用于注册和解除注册。彼此状态的传递使用的也是实例方法，而不是继承。
- 类和类尽量解耦：同2.